# Colab上でVQA-competitionを実行する

Google Colab上で [VQA-competition](https://github.com/ailorg/dl_lecture_competition_pub/tree/VQA-competition) を実行するノートブックの例です。

* 単に「なんかこうやったらうまくうごいた」という共有なので、ノークレームでお願いします。
* 個別のご質問にはお答えしかねます。
* 実行時間等については 2024年6月時点の参考値です。時期やコンピューターの気分によって変わると思うので参考程度にしてください。

## 前提やら基礎知識やら

### Google Colab 関連

* Google Colab には VQA-competition で必要になるライブラリー (torch など) が事前にインストールされていますが、予告なくバージョンが変わるので、昨日まで動いてたのに今日は動かない、ということがありえます。それはそういうもので天災みたいなものなのでノークレームでお願いします。
    * 現実問題としては、そこまで凝った使い方をしていなければまず動かなくなることはないとは思うので、深刻にリスクとして扱うほどの必要はないとは思う。
* クラウド系のサービスは突然画面デザインが変わるのが常なので、ここの説明と画面が一致しない場合があります。その場合はなんとかがんばってください。ノークレームでお願いします。
* Google Colab で実際に処理が動くコンピューターのことをランタイムといいます。基本的にランタイムをずっと維持することはできない(と思うけどよく知らん)ので、必要なデータは Google Drive に退避して、毎回最初に Google Drive からデータを復元して使うという運用になります。
    * 例えば訓練データセットを毎回 Google Drive からランタイムに展開して使う必要がある。
* Google Colab でランタイムを連続で利用できる時間には制限があるため、学習処理を完遂する前にランタイムの期限が切れることがあります。特に Google Colab の無料版だと深刻です。ここは何らかの工夫をする必要がある点に注意してください。
    * 何らかの工夫って何よ？と聞かれても私も困っている立場なのでなんとも言えません。むしろ教えて下さい。

### Google Drive 関連

* Google Drive 上で直接訓練データを展開して Colab からアクセスすると、学習処理が非常に遅くなるという報告があるので注意してください。
* このため、このノートブックでは Google Drive には zip ファイルだけおいて、学習前に Google Colab のランタイム上に展開する、という手順を取ります。

### GitHub 関連

* GitHub からのファイルの取得であれば特に認証の設定は必要ないので、その方法を取ります。
    * リポジトリをパブリックにしている場合に限る。ただし今回、パブリックにしないということはないと思う。
* 想定運用手順としては、 PC 上でソースコードを編集 → GitHub に保存 → Colab から更新したコードを GitHub から取得 → 実行 のサイクルになります。
* Colab から直接 GitHub に保存したい、という方は認証の設定を追加してください。
    * 私はやり方を知らない。

## 事前の準備

### 訓練データ・検証データの準備

* [データのダウンロード](https://github.com/ailorg/dl_lecture_competition_pub/blob/VQA-competition/README.md#%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89) にあるデータを Colab からアクセスできるようにします。
* Colab からは自分の Google Drive にしかアクセスできませんが、このデータを自分の Google Drive にコピーすると容量を圧迫するため、「ショートカットを追加」を行い、自分の Google Drive 経由で直接配布元のファイルにアクセスできるようにします。

Colabへのスクショの貼り方が分からないので言葉だけで説明するのでうまく読み取ってください:

1. [「こちらから各データをダウンロードしてください．」のリンク](https://drive.google.com/drive/folders/1QTcWMATZ_iGsHnxq6-3aXa7D5VZAzs5T?usp=sharing) を開く
2. 「共有アイテム > VQA」の VQA をクリック、整理 > ショートカットを追加 をクリック。
3. 「すべての場所」タブの「マイドライブ」を選んで「追加」
    * もちろん他の場所でも良い。このへんは趣味で。
    * ここではマイドライブ直下に作った前提とします。他の場所に作った方は適宜読み替えてください。
4. マイドライブを見ると「VQA」というフォルダーがあって、なんかフォルダーのアイコンに矢印が出ているはず。

### 自分の環境に合わせて設定を書き換える

1つ下のセルの内容を自分の環境に合わせて書き換えてください。



In [1]:
# ここの URL をご自分がフォークした GitHub リポジトリーの URL に変更してください。(とりあえず動かす分にはこのままでも動くよ)
repo = "https://github.com/ailorg/dl_lecture_competition_pub.git"
branch = "VQA-competition"
repodir = "dl_lecture_competition_pub"
# Googleドライブのマイドライブ以下のフォルダー
# 「ショートカットの追加」でショートカットを作った場所
datadir = "VQA"
# 処理結果を保存する場所
outputdir = "VQA-output"

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
! [ -d $repodir ] || git clone $repo
! cd $repodir && git checkout -B $branch origin/$branch

Cloning into 'dl_lecture_competition_pub'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 55 (delta 3), reused 0 (delta 0), pack-reused 42
Receiving objects: 100% (55/55), 62.46 KiB | 15.61 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Branch 'VQA-competition' set up to track remote branch 'VQA-competition' from 'origin'.
Switched to a new branch 'VQA-competition'


In [5]:
# 訓練データ・検証データを展開する。
# 5分くらいかかります。気長に待ってください。
# ちなみに zip ファイルをランタイムにコピーしてから unzip も試してみたけれど特に早くならんかった(むしろ遅くなるように感じた)。
! [ -f $repodir/data/train.json ] || cp /content/drive/MyDrive/$datadir/train.json $repodir/data/
! [ -f $repodir/data/valid.json ] || cp /content/drive/MyDrive/$datadir/valid.json $repodir/data/
# ! [ -f $repodir/data/train.zip ] || cp /content/drive/MyDrive/$datadir/train.zip $repodir/data/
# ! [ -f $repodir/data/valid.zip ] || cp /content/drive/MyDrive/$datadir/valid.zip $repodir/data/
# ! [ -d $repodir/data/train ] || unzip -q $repodir/data/train.zip -d $repodir/data/
# ! [ -d $repodir/data/valid ] || unzip -q $repodir/data/valid.zip -d $repodir/data/
! [ -d $repodir/data/train ] || unzip -q /content/drive/MyDrive/$datadir/train.zip -d $repodir/data/
! [ -d $repodir/data/valid ] || unzip -q /content/drive/MyDrive/$datadir/valid.zip -d $repodir/data/

In [6]:
# コードを最新化
# コードを更新した場合はここから実行する
# ファイルを書き換えている場合、巻き戻るので注意。
! cd $repodir && git checkout -f
! cd $repodir && git fetch origin $branch && git checkout -B $branch origin/$branch

Your branch is up to date with 'origin/VQA-competition'.
From https://github.com/ailorg/dl_lecture_competition_pub
 * branch            VQA-competition -> FETCH_HEAD
Branch 'VQA-competition' set up to track remote branch 'VQA-competition' from 'origin'.
Reset branch 'VQA-competition'
Your branch is up to date with 'origin/VQA-competition'.


In [7]:
# 処理を無理やり短時間で終わらせるためのハック
# 訓練データ・検証データを一部だけしか使わない。(1000件で1エポック30秒強くらいになる)
# 5エポックだけ学習する。
# 動作確認時にアンコメントして使う。当たり前だけれど最終的な動作としてはここを動かすと全く意味がないので注意。
# ! cd $repodir && git diff
# ! sed -i -e 's/len(self.df)/1000/' -e 's/num_epoch = 20/num_epoch = 5/' $repodir/main.py
# ! cd $repodir && git diff

diff --git a/main.py b/main.py
index 5c6e5ab..0f6c588 100644
--- a/main.py
+++ b/main.py
@@ -148,7 +148,7 @@ class VQADataset(torch.utils.data.Dataset):
             return image, torch.Tensor(question)
 
     def __len__(self):
-        return len(self.df)
+        return 1000
 
 
 # 2. 評価指標の実装
@@ -378,7 +378,7 @@ def main():
     model = VQAModel(vocab_size=len(train_dataset.question2idx)+1, n_answer=len(train_dataset.answer2idx)).to(device)
 
     # optimizer / criterion
-    num_epoch = 20
+    num_epoch = 5
     criterion = nn.CrossEntropyLoss()
     optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
 


In [8]:
# 初期の実装で1エポック10分くらいかかります。10分間なんの反応もなくて不安になりますが、そういうものです。
!cd $repodir && python3 main.py

【1/5】
train time: 33.41 [s]
train loss: 8.2464
train acc: 0.4225
train simple acc: 0.3440
【2/5】
train time: 31.33 [s]
train loss: 5.1783
train acc: 0.4832
train simple acc: 0.3947
【3/5】
train time: 28.44 [s]
train loss: 4.3816
train acc: 0.4824
train simple acc: 0.3933
【4/5】
train time: 29.08 [s]
train loss: 4.1828
train acc: 0.4828
train simple acc: 0.3947
【5/5】
train time: 28.69 [s]
train loss: 4.0167
train acc: 0.4827
train simple acc: 0.3940


In [9]:
# 提出用ファイルの保存
!cp $repodir/model.pth /content/drive/MyDrive/$outputdir
!cp $repodir/submission.npy /content/drive/MyDrive/$outputdir